In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

In [2]:
df=pd.read_excel('prediction_dataset.xlsx')
df.head()

,customer_id,gender,age,customer_state,txn_month,txn_week,txn_day,amount,status,txn_description,movement,balance
0,CUS-1005756958,F,53,QLD,8,W1,2,6.48,authorized,POS,debit,463.96
1,CUS-1005756958,F,53,QLD,8,W1,2,39.95,authorized,SALES-POS,debit,424.01
2,CUS-1005756958,F,53,QLD,8,W1,3,22.96,authorized,POS,debit,401.05
3,CUS-1005756958,F,53,QLD,8,W1,3,9.11,authorized,POS,debit,391.94
4,CUS-1005756958,F,53,QLD,8,W1,3,14.65,authorized,SALES-POS,debit,377.29


In [3]:
df2=df.loc[df['movement']=='debit']
df2=df2.groupby(['customer_id']).sum().reset_index()[['customer_id','amount']]
df2['spending_per_month']=df2['amount']/3
df2=df2.drop('amount',axis=1)
df2.head()

,customer_id,spending_per_month
0,CUS-1005756958,1217.620000
1,CUS-1117979751,2977.940000
2,CUS-1140341822,1837.180000
3,CUS-1147642491,2244.250000
4,CUS-1196156254,2908.203333


In [4]:
df3=df.loc[df['txn_description']=='PAY/SALARY']
y=df3['amount']
df3=df3.iloc[:,:3]
df3['salary']=y
df3=df3.drop_duplicates()
df3.head()

,customer_id,gender,age,salary
14,CUS-1005756958,F,53,970.47
78,CUS-1117979751,M,21,3578.65
177,CUS-1140341822,M,28,1916.51
253,CUS-1147642491,F,34,1711.39
372,CUS-1196156254,F,34,3903.73


In [5]:
df4=pd.merge(df2,df3,on='customer_id').drop('customer_id',axis=1)
df4

,spending_per_month,gender,age,salary
0,1217.620000,F,53,970.47
1,2977.940000,M,21,3578.65
2,1837.180000,M,28,1916.51
3,2244.250000,F,34,1711.39
4,2908.203333,F,34,3903.73
...,...,...,...,...
95,911.510000,F,35,725.32
96,1776.060000,F,21,1037.07
97,2416.053333,M,30,1808.62
98,952.856667,M,26,1433.98


In [6]:
scale_mapper = {'F':1, 
                'M':0}
# Map feature values to scale
df4['gender'] = df4['gender'].replace(scale_mapper)
df4.head()

,spending_per_month,gender,age,salary
0,1217.620000,1,53,970.47
1,2977.940000,0,21,3578.65
2,1837.180000,0,28,1916.51
3,2244.250000,1,34,1711.39
4,2908.203333,1,34,3903.73


In [7]:
scaler = preprocessing.StandardScaler()
x=df4[['spending_per_month','age']]

df5 = scaler.fit_transform(np.array(x, dtype = float))
df5 = pd.DataFrame(df5,columns=x.columns)
df5['gender']=df4['gender']
df5['salary']=df4['salary']
df5.head()

,spending_per_month,age,gender,salary
0,-0.851339,1.848275,1,970.47
1,1.179128,-0.937632,0,3578.65
2,-0.136698,-0.328215,0,1916.51
3,0.332842,0.194143,1,1711.39
4,1.098689,0.194143,1,3903.73


In [8]:
df5.corr()

,spending_per_month,age,gender,salary
spending_per_month,1.000000,-0.086176,0.065956,0.103132
age,-0.086176,1.000000,-0.012067,-0.135264
gender,0.065956,-0.012067,1.000000,-0.154113
salary,0.103132,-0.135264,-0.154113,1.000000


In [9]:
df4.to_csv('unscaled_data.csv',index=False)
df5.to_csv('scaled_data.csv',index=False)